# Filtre des données SWOT - Tutoriel / SWOT Data Filter - Tutorial

Ce projet a pour objectif de rendre les données de la mission SWOT plus accessibles en développant un script en Python qui filtre et convertit les données brutes de la NASA de format netCDF vers des fichiers TIFF, lesquels sont plus légers et conviviaux. Le filtrage des données permet de conserver uniquement les données de bonne qualité pour les analyses. Nous avons utilisé le produit suivant de la mission SWOT pour ce tutoriel: 'SWOT Level 2 Water Mask Raster Image Data Product, Version C', disponible sur le site Earth Data de la NASA (https://search.earthdata.nasa.gov/search?q=SWOT_). Il est à noter qu'il nécessaire de se créer un compte Earth Data afin de pouvoir télécharger les images. Vous pouvez utiliser ce script dans Google Colab (à partir de votre compte Gmail).

This project aims to make the data from the SWOT mission more accessible by developing a Python script that filters and converts raw NASA data from the netCDF format into TIFF files, which are lighter and more user-friendly. Data filtering ensures that only good quality data is retained for analysis. We used the following product from the SWOT mission for this tutorial: 'SWOT Level 2 Water Mask Raster Image Data Product, Version C', available on NASA's Earth Data site (https://search.earthdata.nasa.gov/search?q=SWOT_). Note that it's necessary to create an Earth Data account to download the images. You can use this script in Google Colab (using your Gmail account)

In [1]:
# Installation des packages et importation des librairies nécessaires.
# Install packages and import required libraries
%%capture
!pip install xarray
!pip install rioxarray
!apt install gdal-bin
import xarray
import numpy as np
import rioxarray

# Importer le fichier .nc dans le projet et convertir ce dernier en geo tiff pour 'wse_qual' (drapeau de qualité pour la donnée d'élévation de la surface de l'eau) et 'wse' (élévation de la surface de l'eau) avec gdal_translate
# Import the .nc file into the project and convert it to GeoTIFF for 'wse_qual' (water surface elevation dquality flag) and 'wse' (water surface elevation) using gdal_translate.
!gdal_translate -of GTiff "NETCDF:****:wse_qual" wse_qual.tif #remplacer **** par le chemin d'accès de l'image / replace **** by the path to the image
!gdal_translate -of GTiff "NETCDF:****:wse" wse.tif

##exemple/example:
#!gdal_translate -of GTiff "NETCDF:/content/SWOT_L2_HR_Raster_100m_UTM22J_N_x_x_x_013_533_051F_20240415T150120_20240415T150141_PIC0_01.nc:wse_qual" output_wse_qual.tif #remplacer par le chemin d'accès de l'image / replace by the path to the image
#!gdal_translate -of GTiff "NETCDF:/content/SWOT_L2_HR_Raster_100m_UTM22J_N_x_x_x_013_533_051F_20240415T150120_20240415T150141_PIC0_01.nc:wse" wse.tif #remplacer par le chemin d'accès de l'image / replace by the path to the image


In [2]:
# Ouverture des fichiers GeoTIFF avec rioxarray
# Open GeoTIFF files with rioxarray
wse_qual = rioxarray.open_rasterio("/content/wse_qual.tif") #water surface elevation quality flag (wse_qual)
wse = rioxarray.open_rasterio("/content/wse.tif") #water surface elevation (wse)

# Conversion de wse_qual en DataFrame
# Convert wse_qual to DataFrame
df = wse_qual.to_dataframe()

# Extraction des valeurs uniques pour les flags de qualité (wes_qual)
# Extract unique values for quality flags (wse_qual)
uniqueFlagValues = df['wse_qual'].unique()
print('unique flag values: ', uniqueFlagValues)

# Afficher la valeur minimale parmi les valeurs uniques des flags de qualité
# Find the minimum value among unique flag values
min_value = min(uniqueFlagValues)
print(f"minimum value: {min_value}")

unique flag values:  [3 1 2 0]
minimum value: 0


explication des 'flags' pour les données 'wse_qual'/ Explanation of flags for wse_qual data:
*   0 = good
*   1 = suspect - may have large errors
*   2 = degraded - likely to have large errors<
*   3 = bad - may be nonsensical and should be ignored

In [3]:
# Créer un masque en fonction de la valeur de wse_qual souhaitée
# Create a mask based on the desired wse_qual value
mask = wse_qual <= 1 # <= 1,2,3,4 ou == 0

# Appliquer le masque à 'wse' pour filtrer les valeurs
# Apply the mask to 'wse' to filter values
wse_filtered = wse.where(mask)

# Enregistrer les données wse filtrées dans un nouveau fichier GeoTIFF
# Save the filtered wse data to a new GeoTIFF file
wse_filtered.rio.to_raster("output_wse_filtered.tif")

# Importer le fichier .tif dans QGIS en lui attribuant la bonne projection UTM en fonction de la région à l'étude.
# Import the .tif file into QGIS and assign the correct UTM projection based on the study area.